# Context Managers i Python (Intro Python Eksamen)

**Formål:** Denne notebook er lavet til eksamen til Python intro

Den viser:
- hvorfor context managers findes (ressourcer + oprydning)
- hvordan `with` virker
- hvad `__enter__` og `__exit__` gør
- praktiske eksempler (fil/CSV/JSON/SQLite)
- (bonus) `contextlib.contextmanager`


## 1) Intro – hvad er en context manager?

En **context manager** er et Python-objekt, der hjælper os med at håndtere ressourcer sikkert.

En ressource kan fx være:
- en fil (skal åbnes/lukkes)
- en databaseforbindelse (skal åbnes/lukkes/commit/rollback)
- en lock, netværksforbindelse, timer, mm.

**Pointen:** Oprydning skal ske **uanset** om der opstår fejl undervejs.

## 2) Problemet uden `with`

Hvis vi selv åbner en ressource, kan vi nemt komme til at glemme oprydning, eller en fejl kan stoppe programmet før oprydningen sker.

In [ ]:
# Eksempel uden context manager (IKKE anbefalet)
f = open("demo.txt", "w", encoding="utf-8")
f.write("Hej verden!")
f.close()

print("Skrev til filen og lukkede den manuelt.")

### Hvis der sker en fejl før `close()`

Så kan filen blive efterladt åben. Man kan håndtere det med `try/finally`, men det bliver hurtigt “støjende”.

In [ ]:
# Try/finally virker, men bliver hurtigt mere kode
f = open("demo2.txt", "w", encoding="utf-8")
try:
    f.write("Dette bliver skrevet, selvom vi har sikker oprydning.")
finally:
    f.close()

print("Filen blev lukket via finally.")

## 3) Løsningen: `with`

`with`-sætningen bruger en context manager til at sikre, at oprydning sker automatisk.

In [ ]:
with open("demo_with.txt", "w", encoding="utf-8") as f:
    f.write("Med with bliver filen altid lukket korrekt.")

print("Filen er lukket automatisk efter with-blokken.")

### Nøglepointe

`with` kan tænkes som syntaktisk sukker for et mønster ala:

- “gå ind i konteksten” (setup)
- kør blokken
- “gå ud af konteksten” (teardown), også ved fejl

## 4) Hvad sker der bag kulissen?

En context manager implementerer typisk to metoder:

- `__enter__()` (kaldes når vi går ind i `with`)
- `__exit__(exc_type, exc_value, traceback)` (kaldes når vi forlader `with`)

Python gør i praksis noget i retning af dette:

```python
mgr = <context manager>
value = mgr.__enter__()
try:
    ...  # with-blokken
finally:
    mgr.__exit__(...)
```

> Det vigtige er at forstå idéen: **setup + teardown**.

## 5) Egen context manager (klasse)

Her er en minimal context manager, der bare printer hvad der sker.

In [1]:
class SimpleContext:
    def __enter__(self):
        print("✅ __enter__: Starter (setup)")
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        print("🧹 __exit__: Slutter (teardown)")
        # Returnerer False/None => exceptions bliver ikke 'spist' (de bobler videre)
        return False

with SimpleContext():
    print("👉 Inde i with-blokken")

✅ __enter__: Starter (setup)
👉 Inde i with-blokken
🧹 __exit__: Slutter (teardown)


### `__exit__` ved fejl

Vis at oprydning stadig sker, selv hvis noget går galt.

In [2]:
try:
    with SimpleContext():
        print("Nu laver vi en fejl...")
        1 / 0
except ZeroDivisionError as e:
    print("💥 Fanget fejl udenfor:", e)

✅ __enter__: Starter (setup)
Nu laver vi en fejl...
🧹 __exit__: Slutter (teardown)
💥 Fanget fejl udenfor: division by zero


## 6) Praktiske eksempler

### 6a) CSV

`csv` læser typisk fra en fil – og her er `with` perfekt.

In [3]:
import csv

# Vi laver en lille CSV-fil først
with open("people.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["name", "age"])
    writer.writerow(["Ada", 27])
    writer.writerow(["Bo", 31])

# Læs den igen
with open("people.csv", newline="", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        print(row)

{'name': 'Ada', 'age': '27'}
{'name': 'Bo', 'age': '31'}


### 6b) JSON

`json.load()` og `json.dump()` bruges ofte sammen med `with open(...)`.

In [4]:
import json

data = {"course": "Intro Python", "topic": "context managers", "ok": True}

with open("data.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

with open("data.json", "r", encoding="utf-8") as f:
    loaded = json.load(f)

print(loaded)

{'course': 'Intro Python', 'topic': 'context managers', 'ok': True}


### 6c) SQLite (database)

SQLite er smart til demoer, fordi det ligger i standardbiblioteket.

En databaseforbindelse kan bruges som context manager: `with sqlite3.connect(...) as conn:`

In [5]:
import sqlite3

with sqlite3.connect("demo.db") as conn:
    cur = conn.cursor()
    cur.execute("CREATE TABLE IF NOT EXISTS points (x INTEGER, y INTEGER)")
    cur.execute("INSERT INTO points (x, y) VALUES (?, ?)", (1, 1))
    cur.execute("INSERT INTO points (x, y) VALUES (?, ?)", (2, 3))
    conn.commit()

with sqlite3.connect("demo.db") as conn:
    cur = conn.cursor()
    cur.execute("SELECT x, y FROM points")
    rows = cur.fetchall()

print("Rows:", rows)

Rows: [(1, 1), (2, 3)]


## 7) Context managers vs. try/finally

Begge kan sikre oprydning, men `with` er:
- mere læsbart
- mindre gentagelse
- mindre risiko for at glemme oprydning

**Eksamenstanke:** Python gør det nemt at skrive korrekt kode, fordi mønstret er standardiseret.

## 8) Bonus: `contextlib.contextmanager` (generator-baseret)

En mere “Pythonisk” måde at lave en context manager uden klasse.

**Ide:**
- kode før `yield` = setup
- kode efter `yield` = teardown



In [6]:
from contextlib import contextmanager

@contextmanager
def timer(label="blok"):
    import time
    start = time.time()
    try:
        yield
    finally:
        end = time.time()
        print(f"⏱️ {label}: {end - start:.6f} sek")

with timer("tung beregning"):
    total = sum(i*i for i in range(200_000))
    print("total:", total)

total: 2666646666700000
⏱️ tung beregning: 0.014243 sek


## 10) Konklusion

- Context managers handler om **setup + teardown**
- `with` gør oprydning automatisk og sikker
- `__enter__` og `__exit__` er kernen
- Bruges hele tiden i praksis (filer, CSV/JSON, databaser)
